# Example 4: Markov Stability on hypergraphs

Here, we implement the methodology defined in https://arxiv.org/pdf/2010.14355.pdf

In [ ]:
# The hypernetx package for hypergraphs: https://github.com/pnnl/HyperNetX
!pip install hypernetx

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import scipy as sc
import scipy.sparse as sp
import itertools

import matplotlib.pyplot as plt

import pygenstability as pgs
from pygenstability import plotting

import hypernetx as hnx
from hypernetx.extras import lesmis as lm

## Synthetic hypergraph 

In [ ]:
edges = {
    0: ('1','2','3','4'),
    1: ('5','6','7','8'),
    2: ('9','10','11','12'),
    3: ('13','14','15','16'), 
    4: ('2','5'),
    5: ('4','7'),
    6: ('2','7'),
    7: ('4','5'),
    8: ('7','13'),
    9: ('8','14'),
    10: ('7','14'),
    11: ('13','8'),
    12: ('3','9'),
    13: ('4','10'),
    14: ('3','10'),
    15: ('4','9'),
    16: ('10','13'),
    17: ('12','15'),
    18: ('10','15'),
    19: ('12','13'),
    20: ('4','13'),
    21: ('7','10')
}

hyper_graph = hnx.Hypergraph(edges)
hnx.draw(hyper_graph)

# Projection of hypergraph

In [ ]:
# construct projected matrix (no hyperedges)
incidence = hyper_graph.incidence_matrix().toarray()    
graph_projection = np.matmul(incidence, incidence.T)
np.fill_diagonal(graph_projection, 0)
graph = sp.csr_matrix(graph_projection)

# construct network object just for plotting
nx_graph = nx.Graph(graph)
pos = nx.spring_layout(nx_graph, weight=None, scale=1)
for u in nx_graph:
    nx_graph.nodes[u]["pos"] = pos[u]


In [ ]:
results_hypergraph_projection = pgs.run(graph, min_time=-1, max_time=2, n_time=50,
                                        constructor='continuous_combinatorial')

In [ ]:
plotting.plot_scan(results_hypergraph_projection, use_plotly=True)

In [ ]:
plotting.plot_scan(results_hypergraph_projection, use_plotly=False)

# Equivalent weighted graph

In [ ]:
# defining equivalent weighted network 

def get_adjacency(H):
    """Construct Laplacian for HyperGraph H.
    
    Args:
        H: Hypernetx hypergraph object
    """
    incidence = H.incidence_matrix().toarray()
    
    # hyperedge adjacency matrix
    C = np.matmul(incidence.T, incidence)
    A = np.matmul(incidence, incidence.T)

    R = np.matmul(incidence, np.matmul(np.diag(np.diag(C)), incidence.T))

    # defining transition matrix
    adj = R - A
    np.fill_diagonal(adj, 0)
    
    return adj

In [ ]:
# adjacency matrix constructed using Carletti method
graph_ew = sp.csr_matrix(get_adjacency(hyper_graph))

In [ ]:
results_hypergraph_ew = pgs.run(graph_ew, min_time=-1, max_time=2, n_time=50,
                                constructor='continuous_combinatorial')

In [ ]:
plotting.plot_scan(results_hypergraph_ew, use_plotly=True)

In [ ]:
plotting.plot_scan(results_hypergraph_ew, use_plotly=False)